The purpose of this notebook is to build a scraper for the LA Marathon data. The results are available at the link below:

https://www.trackshackresults.com/lamarathon/results/2017/

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# Url that we are scraping. The numbers give the age range (inclusive), and the _m means male
url_15_under_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=D&Ind=0"
url_16_19_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=DA&Ind=1"
url_20_24_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=E&Ind=2"
url_25_29_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=F&Ind=3"
url_30_34_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=G&Ind=4"
url_35_39_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=H&Ind=5"
url_40_44_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=I&Ind=6"
url_45_49_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=J&Ind=7"
url_50_54_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=K&Ind=8"
url_55_59_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=L&Ind=9"
url_60_64_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=M&Ind=10"
url_65_69_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=MA&Ind=11"
url_70_74_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=N&Ind=12"
url_75_79_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=NA&Ind=13"
url_80_over_m = "https://www.trackshackresults.com/lamarathon/results/2017/mar_results.php?Link=9&Type=2&Div=NB&Ind=14"

url_list = [url_15_under_m, url_16_19_m, url_20_24_m, url_25_29_m, url_30_34_m, url_35_39_m, url_40_44_m, \
           url_45_49_m, url_50_54_m, url_55_59_m, url_60_64_m, url_65_69_m, url_70_74_m, url_75_79_m, url_80_over_m]

html_list = [urlopen(url) for url in url_list]

soup_list = [BeautifulSoup(html, 'html.parser') for html in html_list]

tr_list = [soup.find_all('tr') for soup in soup_list]

results_list = [tr[11:] for tr in tr_list] # The first 11 rows are junk

In [3]:
columns = ['div_place', 'name', 'bib', 'age', 'place', 'gender_place', '5k_split', '10k_split', '15k_split', \
           '20k_split', '25k_split', '30k_split', '35k_split', '40k_split', 'clock_time', 'net_time', 'hometown']

athlete_data = []

for division in results_list: # Looks at the particular division...
    for athlete in division: # ...now look at the particular row (athlete) in that division...
        athlete_tr = athlete.find_all('td')[:-1]  #...then find all their td's (holds their data) except the last (an image)
        column_counter = 0
        data_dict = {}
        for td in athlete_tr: # For each bit of data...
            data_dict[columns[column_counter]] = td.string #...assign it to the appropriate key in the dictionary...
            column_counter += 1
        athlete_data.append(data_dict) #...then append this dictionary to the list of athlete data

In [4]:
len(athlete_data)

10951

In [5]:
df = pd.DataFrame(athlete_data, columns=columns)

In [6]:
df.head()

,div_place,name,bib,age,place,gender_place,5k_split,10k_split,15k_split,20k_split,25k_split,30k_split,35k_split,40k_split,clock_time,net_time,hometown
0,1,EDGAR YAK DE PADUA,41626,14,472,445,22:22,44:42,1:07:02,1:29:28,1:53:14,2:18:56,2:46:15,3:12:13,3:42:24,3:23:00,"LOS ANGELES, CA"
1,2,DAVID ROMAN,40554,15,505,473,24:47,47:14,1:08:55,1:31:25,1:54:21,2:18:44,2:45:17,3:13:01,3:33:15,3:24:01,"LOS ANGELES, CA"
2,3,RONALD PINEDA,41123,13,549,509,22:50,45:34,1:08:11,1:30:38,1:54:24,2:20:05,2:47:27,3:13:43,3:44:13,3:25:58,"LOS ANGELES, CA"
3,4,ROBERT HAWRYLO,41574,15,646,596,25:34,48:26,1:11:30,1:34:52,1:59:06,2:24:52,2:51:44,3:17:43,3:39:30,3:28:48,"VAN NUYS, CA"
4,5,BRYAN SON,40561,14,711,654,25:05,50:05,1:14:11,1:37:55,2:02:00,2:26:47,2:53:01,3:19:57,3:35:36,3:30:29,"LOS ANGELES, CA"


In [7]:
df.to_csv('data/2017/mens_results.csv', index=False)

In [8]:
df.isnull().sum()

div_place         0
name              0
bib               0
age               0
place             0
gender_place      0
5k_split        159
10k_split       104
15k_split       151
20k_split       113
25k_split       253
30k_split       163
35k_split       158
40k_split       145
clock_time        0
net_time          0
hometown         57
dtype: int64

In [9]:
time_split_cols = ['5k_split', '10k_split', '15k_split', '20k_split', '25k_split', \
                  '30k_split', '35k_split', '40k_split']

In [10]:
missing_split_mask = df[time_split_cols].isnull().any(axis=1)
missing_split_df = df[missing_split_mask]
full_data_df = df[-missing_split_mask]

In [11]:
def time_in_minutes(time):
    """
    Takes a string in the format h:m:s and converts it to minutes (including decimals)
    """
    hours = 0
    minutes = 0
    seconds = 0
    
    split_time = time.split(':')
    if len(split_time) == 1: # Only seconds...
        seconds = int(split_time)
    elif len(split_time) == 2: # Minutes and seconds...
        minutes = int(split_time[0])
        seconds = int(split_time[1])
    elif len(split_time) == 3: # Hours, Minutes and seconds...
        hours = int(split_time[0])
        minutes = int(split_time[1])
        seconds = int(split_time[2])
    time_in_minutes = 60*hours + minutes + seconds/60
    return time_in_minutes

time_in_minutes = np.vectorize(time_in_minutes)

In [12]:
full_data_df[time_split_cols] = full_data_df[time_split_cols].apply(time_in_minutes)

C:\Users\psavala\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [13]:
full_data_df['clock_time'] = full_data_df['clock_time'].apply(time_in_minutes)
full_data_df['net_time'] = full_data_df['net_time'].apply(time_in_minutes)

C:\Users\psavala\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\psavala\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
missing_split_df.to_csv('data/2017/mens_missing_splits.csv', index=False)
full_data_df.to_csv('data/2017/mens_all_splits_processed.csv', index=False)